In [1]:
import os
from src.gender import gender

In [2]:
import pandas as pd

In [3]:
df = pd.read_json(os.path.join("data", "filmy.json"))

In [4]:
df["Hrají"]

0                                                    None
1                                                    None
2                                                    None
3                                                    None
4                                                    None
                              ...                        
4520    [Igor Chmela (26627), Jana Plodková (24916), C...
4521    [Anna Polívková (15610), Jiří Langmajer (2981)...
4522                                                   []
4523                                                 None
4524                               [Jiří Ployhar (17086)]
Name: Hrají, Length: 4525, dtype: object

In [5]:
df["Hlavní role"] = df["Hrají"].dropna().apply(lambda x: x[:3])

In [6]:
df["Vedlejší role"] = df["Hrají"].dropna().apply(lambda x: x[3:])

## Gender hlavních rolí

In [7]:
def gender_hlavnich_roli(x):

    if not isinstance(x, list):
        return x
    
    muz_item = []
    zena_item = []
    
    for item in x:
        
        item_stripped = item.split("(")[0].strip()
        gender_result = gender(item_stripped)
        if gender_result == "muž":
            muz_item.append(item)
        elif gender_result == "žena":
            zena_item.append(item)
            
    return [muz_item, zena_item]

In [8]:
df["Muži v hlavních rolích"] = df["Hlavní role"].dropna().apply(lambda x: gender_hlavnich_roli(x)[0])
df["Ženy v hlavních rolích"] = df["Hlavní role"].dropna().apply(lambda x: gender_hlavnich_roli(x)[1])

In [9]:
df.explode("Muži v hlavních rolích").explode("Ženy v hlavních rolích").groupby(["Muži v hlavních rolích","Ženy v hlavních rolích"])["Film"].nunique().nlargest(10)

Muži v hlavních rolích     Ženy v hlavních rolích     
Karel Lamač (37847)        Anny Ondráková (127280)        17
Theodor Pištěk (37366)     Antonie Nedošinská (127295)    12
                           Anny Ondráková (127280)        10
V. Ch. Vladimírov (38066)  Suzanne Marwille (127373)       8
Otto Šimánek (35353)       Veronika Renčová (128659)       7
Theodor Pištěk (37366)     Máňa Ženíšková (126674)         7
Jan Werich (1287)          Veronika Renčová (128659)       6
Karel Roden (494)          Ivana Chýlková (4684)           6
Hugo Haas (37289)          Adina Mandlová (55611)          5
                           Antonie Nedošinská (127295)     5
Name: Film, dtype: int64

In [10]:
df[df["Copyright"] > 1945].explode("Muži v hlavních rolích").explode("Ženy v hlavních rolích").groupby(["Muži v hlavních rolích","Ženy v hlavních rolích"])["Film"].nunique().nlargest(35)

Muži v hlavních rolích      Ženy v hlavních rolích     
Otto Šimánek (35353)        Veronika Renčová (128659)      7
Jan Werich (1287)           Veronika Renčová (128659)      6
Karel Roden (494)           Ivana Chýlková (4684)          6
Jaromír Hanzlík (14425)     Daniela Kolářová (1847)        5
Vlastimil Brodský (833)     Jana Brejchová (7710)          5
Gustáv Valach (10715)       Jana Brejchová (7710)          4
Jiří Schmitzer (2936)       Helena Vitovská (34062)        4
Jiří Sovák (5288)           Iva Janžurová (1049)           4
Tomáš Holý (27091)          Jana Brejchová (7710)          4
Jan Kačer (1165)            Jana Brejchová (7710)          3
Jiří Bartoška (827)         Ivana Chýlková (4684)          3
Jiří Sovák (5288)           Jana Hlaváčová (11370)         3
Josef Abrhám (2274)         Libuše Šafránková (2272)       3
Josef Šebánek (69908)       Marie Motlová (35339)          3
Karel Höger (38146)         Dana Medřická (64028)          3
                            J

## První muž v titulcích + první žena v titulcích

In [11]:
def hlavni_hvezdy(x):

    if not isinstance(x, list):
        return x
    
    muz_item = None
    zena_item = None
    
    for item in x:
        
        item_stripped = item.split("(")[0].strip()
        gender_result = gender(item_stripped)
        if gender_result == "muž" and not muz_item:
            muz_item = item
        elif gender_result == "žena" and not zena_item:
            zena_item = item
        
        if muz_item and zena_item:
            break
    
    return [muz_item, zena_item]

In [12]:
pokus = ["Igor Chmela (26627)", "Jana Plodková (24916)"]

In [13]:
hlavni_hvezdy(pokus)

['Igor Chmela (26627)', 'Jana Plodková (24916)']

In [14]:
df["Muž v hlavní roli"] = df["Hrají"].dropna().apply(lambda x: hlavni_hvezdy(x)[0])
df["Žena v hlavní roli"] = df["Hrají"].dropna().apply(lambda x: hlavni_hvezdy(x)[1])

In [15]:
df.groupby(["Muž v hlavní roli","Žena v hlavní roli"])["Film"].nunique().nlargest(10)

Muž v hlavní roli          Žena v hlavní roli         
Karel Lamač (37847)        Anny Ondráková (127280)        12
Theodor Pištěk (37366)     Antonie Nedošinská (127295)    12
Otto Šimánek (35353)       Veronika Renčová (128659)       7
V. Ch. Vladimírov (38066)  Suzanne Marwille (127373)       6
Theodor Pištěk (37366)     Anny Ondráková (127280)         5
Zdeněk Štěpánek (87457)    Vlasta Matulová (998)           5
Hugo Haas (37289)          Antonie Nedošinská (127295)     4
Jindřich Plachta (5829)    Zdeňka Baldová (28798)          4
Jiří Schmitzer (2936)      Helena Vitovská (34062)         4
Karel Noll (122320)        Antonie Nedošinská (127295)     4
Name: Film, dtype: int64

In [16]:
df[df["Copyright"] > 1945].groupby(["Muž v hlavní roli","Žena v hlavní roli"])["Film"].nunique().nlargest(20)

Muž v hlavní roli           Žena v hlavní roli         
Otto Šimánek (35353)        Veronika Renčová (128659)      7
Jiří Schmitzer (2936)       Helena Vitovská (34062)        4
Vlastimil Brodský (833)     Jana Brejchová (7710)          4
Zdeněk Štěpánek (87457)     Vlasta Matulová (998)          4
Gustáv Valach (10715)       Jana Brejchová (7710)          3
Jan Kačer (1165)            Jana Brejchová (7710)          3
Jiří Sovák (5288)           Jiřina Bohdalová (2878)        3
Josef Abrhám (2274)         Libuše Šafránková (2272)       3
Josef Šebánek (69908)       Marie Motlová (35339)          3
Karel Heřmánek (6274)       Jana Synková (12616)           3
Karel Höger (38146)         Dana Medřická (64028)          3
                            Slávka Budínová (8606)         3
Martin Finger (19835)       Anna Geislerová (5433)         3
Otomar Krejča (7597)        Antonie Hegerliková (7721)     3
Otto Šimánek (35353)        Slávka Budínová (8606)         3
Radoslav Brzobohatý (2770)  J

In [17]:
df[df["Copyright"] > 1989].groupby(["Muž v hlavní roli","Žena v hlavní roli"])["Film"].nunique().nlargest(20)

Muž v hlavní roli          Žena v hlavní roli         
Karel Heřmánek (6274)      Jana Synková (12616)           3
Martin Finger (19835)      Anna Geislerová (5433)         3
Václav Vydra (2666)        Jana Paulová (8262)            3
Bolek Polívka (1556)       Eva Holubová (5948)            2
Boleslav Polívka (1556)    Eva Holubová (5948)            2
                           Veronika Žilková (1517)        2
                           Zlata Adamovská (18668)        2
                           Zuzana Bydžovská (4683)        2
Jaromír Dulava (1837)      Marie Mazurová (4266)          2
Jiří Bartoška (827)        Ivana Chýlková (4684)          2
Jiří Langmajer (2981)      Anna Polívková (15610)         2
Jiří Schmitzer (2936)      Helena Vitovská (34062)        2
Josef Abrhám (2274)        Libuše Šafránková (2272)       2
Karel Roden (494)          Anna Geislerová (5433)         2
                           Chantal Poullain /ž/ (1573)    2
                           Lenka Vlasáková (1

In [18]:
df[df["Copyright"] > 2002].groupby(["Muž v hlavní roli","Žena v hlavní roli"])["Film"].nunique().nlargest(20)

Muž v hlavní roli         Žena v hlavní roli      
Martin Finger (19835)     Anna Geislerová (5433)      3
Václav Vydra (2666)       Jana Paulová (8262)         3
Bolek Polívka (1556)      Eva Holubová (5948)         2
Boleslav Polívka (1556)   Eva Holubová (5948)         2
                          Zlata Adamovská (18668)     2
Jiří Langmajer (2981)     Anna Polívková (15610)      2
Martin Myšička (13007)    Jana Plodková (24916)       2
Martin Pechlát (9426)     Gabriela Míčová (9758)      2
Oldřich Kaiser (1794)     Kamila Magálová (55419)     2
Ondřej Vetchý (836)       Anna Geislerová (5433)      2
                          Vica Kerekes /ž/ (78381)    2
Vratislav Brabenec (663)  Nikol Fischerová (2527)     2
Václav Vydra (2666)       Dana Morávková (2293)       2
Aaron Monaghan (59068)    Réka Derzsi /ž/ (35476)     1
Adam Gebert (19050)       Anna Lísalová (78487)       1
Adam Mihál (122066)       Mária Fornayová (130007)    1
Adam Mišík (27584)        Anna Fialová (21871)       

In [19]:
df.groupby("Žena v hlavní roli")["Film"].nunique().nlargest(20)

Žena v hlavní roli
Antonie Nedošinská (127295)    41
Jana Brejchová (7710)          37
Suzanne Marwille (127373)      35
Anny Ondráková (127280)        31
Iva Janžurová (1049)           29
Adina Mandlová (55611)         26
Hana Vítová (126418)           26
Jiřina Bohdalová (2878)        25
Anna Geislerová (5433)         23
Jiřina Štěpničková (29765)     23
Ivana Chýlková (4684)          21
Zdeňka Baldová (28798)         21
Lída Baarová (5202)            19
Jiřina Šejbalová (125559)      18
Květa Fialová (5338)           17
Libuše Šafránková (2272)       17
Slávka Budínová (8606)         17
Dana Medřická (64028)          16
Růžena Šlemrová (127282)       16
Věra Ferbasová (33991)         16
Name: Film, dtype: int64

In [20]:
df.explode("Režie").groupby(["Režie","Žena v hlavní roli"])["Film"].nunique().nlargest(20)

Režie                       Žena v hlavní roli         
Václav Binovec (37467)      Suzanne Marwille (127373)      21
Karel Lamač (37847)         Anny Ondráková (127280)        17
Max Urban (95056)           Andula Sedláčková (126987)     10
Vladimír Slavínský (31967)  Antonie Nedošinská (127295)    10
                            Věra Ferbasová (33991)          9
Jindřich Polák (7891)       Veronika Renčová (128659)       7
Martin Frič (20324)         Adina Mandlová (55611)          7
                            Suzanne Marwille (127373)       7
Jan Hřebejk (4831)          Anna Geislerová (5433)          6
Svatopluk Innemann (33860)  Zdena Kavková (127457)          6
Martin Frič (20324)         Běla Jurdová (27291)            5
                            Zdeňka Baldová (28798)          5
Miroslav Cikán (31969)      Hana Vítová (126418)            5
Oldřich Kmínek (37986)      Anita Janová (128156)           5
Otakar Vávra (9507)         Vlasta Matulová (998)           5
Evald Schorm (

In [21]:
df.explode("Režie").groupby(["Režie","Muž v hlavní roli"])["Film"].nunique().nlargest(20)

Režie                       Muž v hlavní roli          
Jindřich Polák (7891)       Otto Šimánek (35353)           16
Martin Frič (20324)         Vlasta Burian (11992)          14
Karel Lamač (37847)         Vlasta Burian (11992)          11
                            Karel Lamač (37847)            10
Martin Frič (20324)         Hugo Haas (37289)               8
Dušan Klein (2568)          Pavel Kříž (2990)               7
Otakar Vávra (9507)         Zdeněk Štěpánek (87457)         7
Martin Frič (20324)         František Smolík (126955)       6
                            Oldřich Nový (12009)            6
Vladimír Slavínský (31967)  Theodor Pištěk (37366)          6
                            Vladimír Slavínský (31967)      6
Václav Binovec (37467)      V. Ch. Vladimírov (38066)       6
Josef Kokeisl (55986)       Roman Roda-Růžička (128234)     5
Karel Lamač (37847)         Theodor Pištěk (37366)          5
Martin Frič (20324)         Zdeněk Štěpánek (87457)         5
Miroslav Cikán

## Hvězdnost herectva

In [22]:
hvezdnost = pd.Series(df.explode("Hlavní role").groupby("Hlavní role")["Film"].nunique() / df.explode("Hrají").groupby("Hrají")["Film"].nunique(), name="Hvězdnost")

In [23]:
hvezdnost

 (person)                        NaN
 Ajala (77698)                   NaN
 Baky (13703)                    NaN
 Bohemia Big Band (18563)        NaN
 Boni Pueri (22837)              NaN
                                ... 
Žofie Veselá (125302)            NaN
Žofie Červená (130187)           NaN
Žák (63328)                      NaN
Žán Loose (11369)                1.0
žáci Scholae cantorum (59846)    NaN
Name: Hvězdnost, Length: 16194, dtype: float64

In [24]:
obsazovanost = pd.Series(df.explode("Hrají").groupby("Hrají")["Film"].nunique(), name="Obsazovanost")

In [25]:
obsazovanost

Hrají
 (person)                         2
 Ajala (77698)                    1
 Baky (13703)                     1
 Bohemia Big Band (18563)         1
 Boni Pueri (22837)               1
                                 ..
Žofie Veselá (125302)            14
Žofie Červená (130187)            1
Žák (63328)                       1
Žán Loose (11369)                 1
žáci Scholae cantorum (59846)     1
Name: Obsazovanost, Length: 16194, dtype: int64

In [26]:
hvezdy = pd.DataFrame([hvezdnost, obsazovanost]).transpose()

In [27]:
hvezdy

,Hvězdnost,Obsazovanost
(person),NaN,2.0
Ajala (77698),NaN,1.0
Baky (13703),NaN,1.0
Bohemia Big Band (18563),NaN,1.0
Boni Pueri (22837),NaN,1.0
...,...,...
Žofie Veselá (125302),NaN,14.0
Žofie Červená (130187),NaN,1.0
Žák (63328),NaN,1.0
Žán Loose (11369),1.0,1.0


In [28]:
hvezdy[hvezdy["Obsazovanost"] >= 15].sort_values(by="Hvězdnost", ascending=False).head(60)

,Hvězdnost,Obsazovanost
Vlasta Burian (11992),0.974359,39.0
Andula Sedláčková (126987),0.937500,16.0
Suzanne Marwille (127373),0.900000,40.0
Rolf Wanka (56226),0.894737,19.0
Anny Ondráková (127280),0.810811,37.0
Hugo Haas (37289),0.766667,30.0
Jan Werich (1287),0.761905,21.0
V. Ch. Vladimírov (38066),0.700000,20.0
Karel Lamač (37847),0.698413,63.0
Karel Roden (494),0.698113,53.0


In [29]:
hvezdy[hvezdy["Obsazovanost"] >= 100].sort_values(by="Hvězdnost", ascending=False).tail(20)

,Hvězdnost,Obsazovanost
Hermína Vojtová (127041),0.072727,110.0
Milka Balek-Brodská (28799),0.066265,166.0
Josef Hlinomaz (26212),0.064748,139.0
František Černý (127313),0.049505,101.0
Jiří Lír (4926),0.037879,132.0
Ota Motyčka (81341),0.025000,120.0
Oldřich Velen (12725),0.024096,166.0
Alois Dvorský (127491),0.017751,169.0
Václav Kotva (2718),0.017699,113.0
Mirko Musil (33555),0.016393,122.0
